In [1]:
import pandas as pd

# # Replace 'your_file.csv' with the path to your CSV file
# df = pd.read_csv('data/Amazon_Reviews_2023.csv')

# # Print the DataFrame
# print(df.head())

# read jsonl file
df = pd.read_json('data/All_Beauty.jsonl', lines=True)
print(df.head())


   rating                                      title  \
0       5  Such a lovely scent but not overpowering.   
1       4     Works great but smells a little weird.   
2       5                                       Yes!   
3       1                          Synthetic feeling   
4       5                                         A+   

                                                text images        asin  \
0  This spray is really nice. It smells really go...     []  B00YQ6X8EO   
1  This product does what I need it to do, I just...     []  B081TJ8YS3   
2                          Smells good, feels great!     []  B07PNNCSP9   
3                                     Felt synthetic     []  B09JS339BZ   
4                                            Love it     []  B08BZ63GMJ   

  parent_asin                       user_id               timestamp  \
0  B00YQ6X8EO  AGKHLEW2SOWHNMFQIJGBECAF7INQ 2020-05-05 14:08:48.923   
1  B081TJ8YS3  AGKHLEW2SOWHNMFQIJGBECAF7INQ 2020-05-04 18:10:55.070   

In [2]:
df.shape
print(df.shape)

total_beauty_products = df.groupby('asin').count()
print(total_beauty_products)

total_uniq_users=df.groupby('user_id')

print(total_uniq_users)

(701528, 10)
            rating  title  text  images  parent_asin  user_id  timestamp  \
asin                                                                       
0005946468       1      1     1       1            1        1          1   
0123034892       1      1     1       1            1        1          1   
0124784577       3      3     3       3            3        3          3   
0515059560       1      1     1       1            1        1          1   
0615675026       1      1     1       1            1        1          1   
...            ...    ...   ...     ...          ...      ...        ...   
B0CBMFK1S2       1      1     1       1            1        1          1   
B0CBWDTY41       2      2     2       2            2        2          2   
B0CBXM7WHY       1      1     1       1            1        1          1   
B0CCPDTRK7       1      1     1       1            1        1          1   
B0CFZKJ4KY       2      2     2       2            2        2          2   

In [3]:
user_counts = df.groupby('user_id').size().reset_index(name='count')

# Sort by count in descending order
sorted_user_counts = user_counts.sort_values(by='count', ascending=False)

# Print the sorted DataFrame
# print(sorted_user_counts.head())
top_user = sorted_user_counts.head(1)['user_id'].values[0]
top_user_data = df[df['user_id'] == top_user]
top_user_reviews = top_user_data['text']

print(top_user_reviews.values)
user_reviews = top_user_reviews.values


# print(df['user_id'].equals(top_user))





['when my daughter and granddaughter come over, they take what I have sitting out. That means my supply of scrunchies diminish and have to be replenished. These may not be able to make two twists for thick hair, but for thin, it will. Maybe even three.'
 'I just put this on and it was creamy and a small amount can go a long way. I even put some on the outer eye wrinkles and the labial lines. I can feel the tightening as I type.'
 'they are not fragrance free though. As know, essential oils have fragrances to them and you can smell the oils in these two bottles. What is bad, is that they both leaked, even though they had plastic covering the tops and on the upper parts of the bottle. They were shipped in a shipper bag with the poly bubble interior and that was not enough protection.'
 'This will help your skin heal quicker and better than just using regular facial products. OxygenCeuticals makes a fine product that is designed to moisturize, calm and repair skin. You will love it after 

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
def analyze_sentiment(text):
    """
    Perform sentiment analysis on the given text using a pretrained BERT model.

    Parameters:
    - text (str): The input text to analyze.

    Returns:
    - dict: Sentiment classification result with label and score.
    """
    # Load the pretrained BERT model for sentiment analysis
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"  # or "distilbert-base-uncased-finetuned-sst-2-english" for English sentiment
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)

    model.to('cpu')
    # Initialize a sentiment analysis pipeline
    sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer,  device=-1)

    # Analyze sentiment
    result = sentiment_pipeline(text)

    return result[0]

# Example usage
text = "I hate using BERT for sentiment analysis; it works great!"
print(analyze_sentiment(text))

/Users/coder/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'label': '5 stars', 'score': 0.62462317943573}


In [5]:
# # Get the top users who provided the most ratings and reviews
# top_users = sorted_user_counts.head(10)
# print("Top users based on number of reviews:")
# print(top_users)

# def generate_custom_description(user_id, product_name):
#     """
#     Generate a customized description for a new product based on the user's past reviews.

#     Parameters:
#     - user_id (str): The user ID.
#     - product_name (str): The name of the new product.

#     Returns:
#     - str: Customized product description.
#     """
#     # Check if 'reviewText' column exists
#     if 'text' not in df.columns:
#         raise KeyError("'text' column is missing from the DataFrame")
    
#     # Get all reviews by the user
#     user_reviews = df[df['user_id'] == user_id]['text']
    
#     # Concatenate all reviews into a single text
#     combined_reviews = " ".join(user_reviews)
    
#     # Truncate the combined reviews to the maximum sequence length supported by the BERT model
#     max_length = 512
#     combined_reviews = combined_reviews[:max_length]
    
#     # Perform sentiment analysis on the combined reviews
#     sentiment_result = analyze_sentiment(combined_reviews)
    
#     # Generate a customized description based on the sentiment
#     if sentiment_result['label'] == 'POSITIVE':
#         description = f"As a top reviewer, you might love our new product, {product_name}! Based on your positive feedback, we believe this product will meet your high standards and expectations."
#     else:
#         description = f"As a top reviewer, we value your feedback on our new product, {product_name}. We hope this product addresses any concerns you had in the past and exceeds your expectations."

#     return description

# # Example usage
# user_id = top_users.iloc[0]['user_id']
# product_name = "New Beauty Product"
# custom_description = generate_custom_description(user_id, product_name)
# print(custom_description)

In [6]:
article = {
    "main_category": "All Beauty",
    "title": "Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)",
    "average_rating": 4.8,
    "rating_number": 10,
    "features": [],
    "description": [],
    "price": 0,
    "images": [
      {
        "thumb": "https://m.media-amazon.com/images/I/41qfjSfqNyL._SS40_.jpg",
        "large": "https://m.media-amazon.com/images/I/41qfjSfqNyL.jpg",
        "variant": "MAIN",
        "hi_res": 0
      },
      {
        "thumb": "https://m.media-amazon.com/images/I/41w2yznfuZL._SS40_.jpg",
        "large": "https://m.media-amazon.com/images/I/41w2yznfuZL.jpg",
        "variant": "PT01",
        "hi_res": "https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg"
      }
    ],
    "videos": [],
    "store": "Howard Products",
    "categories": [],
    "details": {
      "Package Dimensions": "7.1 x 5.5 x 3 inches; 2.38 Pounds",
      "UPC": "617390882781"
    },
    "parent_asin": "B01CUPMQZE",
    "bought_together": 0
  }

product_details = {}
product_details['product_name'] = article['title']
product_details['brand'] = article['store']
product_details['primary_features'] = str(article['features'])
product_details['brand_info'] = "N/A"
product_details['intended_use'] = article['main_category']
product_details['technical_specs'] = str(article['details'])
product_details['aesthetic_qualities'] = "N/A"

# product_details = {
#     "product_name": "XYZ Smart Thermostat",
#     "brand": "EcoLiving",
#     "primary_features": ["energy-efficient", "remote control", "easy installation"],
#     "brand_info": "Known for eco-friendly, innovative products",
#     "intended_use": "home climate control",
#     "technical_specs": "Works with Alexa/Google Assistant, real-time temperature adjustments",
#     "aesthetic_qualities": "sleek, minimalist design in black or white",
# }

In [7]:
# import gensim
# import langchain
# from gensim.models import Word2Vec
# from langchain import LangChain

# def preprocess_reviews(reviews):
#     processed_reviews = [gensim.utils.simple_preprocess(review) for review in reviews]
#     return processed_reviews

# langchain = LangChain()

# # Generate product description based on user reviews
# def generate_product_description(product_name, reviews, word2vec_model):
#     # Use LangChain to generate a description
#     description = langchain.generate_description(product_name, reviews, word2vec_model)
#     return description



In [8]:
# processed_reviews = preprocess_reviews(reviews)
# # Train Word2Vec model
# word2vec_model = Word2Vec(sentences=processed_reviews, vector_size=100, window=5, min_count=1, workers=4)

In [9]:
# %pip install langchain
# %pip install openai
# %pip install langchain_community


In [10]:
# import json
# from langchain.chains import LLMChain
# from langchain.llms import OpenAI
# from langchain.prompts import PromptTemplate

# # Example JSON data
# product_json = '''
# {
#     "product_name": "SuperWidget",
#     "reviews": [
#         "I love this product! It works great.",
#         "Not bad, but could be better.",
#         "Terrible experience, would not recommend."
#     ]
# }
# '''

# # Load the JSON data
# product_details = json.loads(product_json)
# product_name = product_details.get('product_name', 'Unknown Product')
# reviews = product_details.get('reviews', [])

# # Initialize OpenAI LLM
# llm = OpenAI(api_key='')  # Replace with your OpenAI API key

# # Define a prompt template for generating product descriptions
# prompt_template = PromptTemplate(
#     input_variables=["product_name", "reviews"],
#     template="Generate a product description for {product_name} based on the following reviews: {reviews}"
# )

# # Create an LLMChain for generating product descriptions
# chain = LLMChain(llm=llm, prompt=prompt_template)

# # Generate product description based on user reviews
# def generate_product_description(product_name, reviews):
#     # Use LLMChain to generate a description
#     description = chain.run(product_name=product_name, reviews=reviews)
#     return description

# # Generate product description
# description = generate_product_description(product_name, reviews)
# print(description)

In [11]:
# import json
# import gensim
# import langchain
# from gensim.models import Word2Vec
# from langchain.chains import LLMChain

# # Install missing module
# from langchain.llms import OpenAI
# from langchain.prompts import PromptTemplate

# def preprocess_reviews(reviews):
#     processed_reviews = [gensim.utils.simple_preprocess(review) for review in reviews]
#     return processed_reviews

# # Initialize OpenAI LLM
# llm = OpenAI(api_key='')  # Replace with your OpenAI API key

# # Define a prompt template for generating product descriptions
# prompt_template = PromptTemplate(
#     input_variables=["product_name", "reviews"],
#     template="Generate a product description for {product_name} based on the following reviews: {reviews}"
# )

# # Create an LLMChain for generating product descriptions
# langchain = LLMChain(llm=llm, prompt=prompt_template)

# # Generate product description based on user reviews
# def generate_product_description(product_name, reviews):
#     # Use LangChain to generate a description
#     description = langchain.run(product_name=product_name, reviews=reviews)
#     return description

# # Example JSON data
# product_json = '''
# {
#     "product_name": "SuperWidget",
#     "reviews": [
#         "I love this product! It works great.",
#         "Not bad, but could be better.",
#         "Terrible experience, would not recommend."
#     ]
# }
# '''

# # Load the JSON data
# product_details = json.loads(product_json)
# product_name = product_details.get('product_name', 'Unknown Product')
# reviews = product_details.get('reviews', [])

# # Generate product description
# description = generate_product_description(product_name, reviews)
# print(description)



In [12]:
# def generate_product_description(product_name, brand, primary_features, brand_info, intended_use, 
#                                  technical_specs, aesthetic_qualities, strengths, common_feedback, 
#                                  drawbacks, overall_sentiment, rating):
#     """
#     Generate a contextual product description based on product details and user reviews.

#     Args:
#     - product_name (str): Name of the product.
#     - brand (str): Name of the brand.
#     - primary_features (list): List of primary product features.
#     - brand_info (str): Information about the brand.
#     - intended_use (str): Description of the intended use of the product.
#     - technical_specs (str): Technical specifications of the product.
#     - aesthetic_qualities (str): Aesthetic description of the product.
#     - strengths (str): Summary of product strengths based on user reviews.
#     - common_feedback (str): Common positive feedback from users.
#     - drawbacks (str): Commonly mentioned drawbacks from user reviews.
#     - overall_sentiment (str): General sentiment from user reviews.
#     - rating (float): Average rating out of 5.

#     Returns:
#     - str: Generated product description.
#     """
    
#     # Step 1: Opening Sentence
#     description = f"The {product_name} by {brand} brings {primary_features[0]}, designed for {intended_use}.\n"
    
#     # Step 2: Key Features Overview
#     description += f"With {', '.join(primary_features)}, this product offers {common_feedback}. Its {aesthetic_qualities} provides both a modern aesthetic and added durability.\n"
    
#     # Step 3: User Experience Summary (Positive Feedback)
#     description += f"Users consistently appreciate {strengths}, noting that it {overall_sentiment}.\n"
    
#     # Step 4: Addressing Drawbacks
#     if drawbacks:
#         description += f"While some users mentioned {drawbacks}, most agree this doesn't detract significantly from the product's overall performance.\n"
    
#     # Step 5: Conclusion
#     description += f"Overall, the {product_name} has earned high marks for its {common_feedback}, making it a top choice for those seeking {intended_use}. With an average rating of {rating}/5, it’s clear this product meets user expectations for {primary_features[1]}.\n"
    
#     return description

# # Example usage
# product_name = "XYZ Smart Thermostat"
# brand = "EcoLiving"
# primary_features = ["energy-efficient technology", "remote control via app", "easy installation"]
# brand_info = "Known for eco-friendly, innovative products"
# intended_use = "home climate control"
# technical_specs = "Works with Alexa/Google Assistant, real-time temperature adjustments"
# aesthetic_qualities = "sleek, minimalist design in black or white"
# strengths = "easy setup, responsive controls, energy savings"
# common_feedback = "functionality and style"
# drawbacks = "initial setup may be tricky for non-tech users"
# overall_sentiment = "high user satisfaction, saves on bills, intuitive app"
# rating = 4.7

# description = generate_product_description(
#     product_name, brand, primary_features, brand_info, intended_use, 
#     technical_specs, aesthetic_qualities, strengths, common_feedback, 
#     drawbacks, overall_sentiment, rating
# )

# print(description)

In [13]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
# Sample product details and user reviews
# product_details = {
#     "product_name": "XYZ Smart Thermostat",
#     "brand": "EcoLiving",
#     "primary_features": ["energy-efficient", "remote control", "easy installation"],
#     "brand_info": "Known for eco-friendly, innovative products",
#     "intended_use": "home climate control",
#     "technical_specs": "Works with Alexa/Google Assistant, real-time temperature adjustments",
#     "aesthetic_qualities": "sleek, minimalist design in black or white",
# }
# user_reviews = [
#     "This thermostat was easy to install and saves a lot on my energy bills.",
#     "I love the remote control feature—it’s very convenient.",
#     "Sleek design and energy-efficient technology make this a great purchase.",
#     "A bit tricky to set up at first, but works perfectly afterward.",
# ]


# from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as stopwords
# stop_words = set(stopwords)


nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')


# Step 1: Preprocess the data
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and lowercase
    tokens = word_tokenize(text)
    return [word for word in tokens if word not in stop_words]

print(product_details.values())
all_text = [str(value) for value in product_details.values()] + list(user_reviews)
all_text = [" ".join(text) if isinstance(text, list) else text for text in all_text]
all_text = [preprocess(text) for text in all_text]

print(all_text)
# Step 2: Train the Word2Vec model
model = Word2Vec(sentences=all_text, vector_size=100, window=5, min_count=1, workers=4)

# Step 3: Function to generate product description using word similarity
def generate_contextual_description(details, reviews, model):
    description = f"The {details['product_name']} by {details['brand']} offers {details['primary_features'][0]}, " \
                  f"designed for {details['intended_use']}.\n"

    # Using word vectors to get similar terms for features and feedback
    features = ', '.join(details['primary_features'])
    description += f"With {features}, this product offers both functionality and style. " \
                   f"Its {details['aesthetic_qualities']} provides a modern aesthetic and added durability.\n"
    
    # Summary of positive feedback based on similar words found in reviews
    positive_terms = []
    for review in reviews:
        tokens = preprocess(review)
        for word in tokens:
            if word in model.wv:
                positive_terms.extend([sim[0] for sim in model.wv.most_similar(word, topn=2)])
    positive_summary = ', '.join(set(positive_terms))
    
    description += f"Users consistently appreciate {positive_summary}, making it highly recommended.\n"
    
    # Drawbacks based on review content, if any
    if "tricky" in positive_terms:
        description += "While some users mentioned it can be a bit tricky to set up initially, " \
                       "most agree it doesn't detract from the product's performance.\n"
    
    # Conclusion
    description += f"Overall, the {details['product_name']} has earned positive reviews for " \
                   f"its {positive_summary}. With its combination of technology and ease of use, " \
                   f"it's an excellent choice for {details['intended_use']}."
    
    return description

# Generate and print the contextual description
contextual_description = generate_contextual_description(product_details, user_reviews, model)
print("New Description:", contextual_description)


[nltk_data] Downloading package punkt_tab to /Users/coder/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/coder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/coder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


dict_values(['Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)', 'Howard Products', '[]', 'N/A', 'All Beauty', "{'Package Dimensions': '7.1 x 5.5 x 3 inches; 2.38 Pounds', 'UPC': '617390882781'}", 'N/A'])
[['Howard', 'LC0008', 'Leather', 'Conditioner', '8Ounce', '4Pack'], ['Howard', 'Products'], [], ['NA'], ['All', 'Beauty'], ['Package', 'Dimensions', '71', 'x', '55', 'x', '3', 'inches', '238', 'Pounds', 'UPC', '617390882781'], ['NA'], ['daughter', 'granddaughter', 'come', 'take', 'I', 'sitting', 'That', 'means', 'supply', 'scrunchies', 'diminish', 'replenished', 'These', 'may', 'able', 'make', 'two', 'twists', 'thick', 'hair', 'thin', 'Maybe', 'even', 'three'], ['I', 'put', 'creamy', 'small', 'amount', 'go', 'long', 'way', 'I', 'even', 'put', 'outer', 'eye', 'wrinkles', 'labial', 'lines', 'I', 'feel', 'tightening', 'I', 'type'], ['fragrance', 'free', 'though', 'As', 'know', 'essential', 'oils', 'fragrances', 'smell', 'oils', 'two', 'bottles', 'What', 'bad', 'leaked', 'even', 'thoug

In [14]:
import spacy
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from gensim.models import Word2Vec
import numpy as np

# Load Spacy model for dependency parsing
nlp = spacy.load("en_core_web_sm")

# Sample data
keywords = ["energy-efficient", "remote control", "easy installation"]
sentiments = ["positive", "positive", "neutral"]

# Sample product details
product_details = {
    "product_name": "XYZ Smart Thermostat",
    "brand": "EcoLiving",
    "primary_features": ["energy-efficient", "remote control", "easy installation"],
    "brand_info": "Known for eco-friendly, innovative products",
    "intended_use": "home climate control",
    "technical_specs": "Works with Alexa/Google Assistant, real-time temperature adjustments",
    "aesthetic_qualities": "sleek, minimalist design in black or white",
}

# Sample user reviews
user_reviews = [
    "This thermostat was easy to install and saves a lot on my energy bills.",
    "I love the remote control feature—it's very convenient.",
    "Sleek design and energy-efficient technology make this a great purchase.",
    "A bit tricky to set up at first, but works perfectly afterward.",
]

# Preprocess reviews for Word2Vec
def preprocess(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

all_text = [preprocess(review) for review in user_reviews]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=all_text, vector_size=100, window=5, min_count=1, workers=4)

# Create a dependency parser function
def extract_keywords(text):
    doc = nlp(text)
    return [token.text for token in doc if token.dep_ in ("amod", "nsubj", "dobj")]

# Extract keywords from reviews
extracted_keywords = [extract_keywords(review) for review in user_reviews]

# Flatten the list of keywords
flattened_keywords = [item for sublist in extracted_keywords for item in sublist]

# Vectorize keywords
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(flattened_keywords)

# Create labels for sentiments
y = np.array([1 if sentiment == "positive" else 0 for sentiment in sentiments])

# Train SGDClassifier
# Adjust labels to match the number of flattened keywords
y_adjusted = np.array([1 if sentiment == "positive" else 0 for sentiment in sentiments for _ in range(len(flattened_keywords) // len(sentiments))])

y_adjusted = np.resize(y_adjusted, len(flattened_keywords))

sgd_model = make_pipeline(CountVectorizer(), SGDClassifier())
sgd_model.fit(flattened_keywords, y_adjusted)

# Generate product description
def generate_product_description(details, keywords, sentiments, word2vec_model, sgd_model):
    description = f"The {details['product_name']} by {details['brand']} offers {details['primary_features'][0]}, " \
                  f"designed for {details['intended_use']}.\n"

    # Using word vectors to get similar terms for features and feedback
    features = ', '.join(details['primary_features'])
    description += f"With {features}, this product offers both functionality and style. " \
                   f"Its {details['aesthetic_qualities']} provides a modern aesthetic and added durability.\n"
    
    # Summary of positive feedback based on similar words found in reviews
    positive_terms = []
    for keyword in keywords:
        if keyword in word2vec_model.wv:
            positive_terms.extend([sim[0] for sim in word2vec_model.wv.most_similar(keyword, topn=2)])
    positive_summary = ', '.join(set(positive_terms))
    
    description += f"Users consistently appreciate {positive_summary}, making it highly recommended.\n"
    
    # Drawbacks based on review content, if any
    if "tricky" in positive_terms:
        description += "While some users mentioned it can be a bit tricky to set up initially, " \
                       "most agree it doesn't detract from the product's performance.\n"
    
    # Conclusion
    description += f"Overall, the {details['product_name']} has earned positive reviews for " \
                   f"its {positive_summary}. With its combination of technology and ease of use, " \
                   f"it's an excellent choice for {details['intended_use']}."
    
    return description

# Generate and print the product description
contextual_description = generate_product_description(product_details, keywords, sentiments, word2vec_model, sgd_model)
print("Generated Description:", contextual_description)

Generated Description: The XYZ Smart Thermostat by EcoLiving offers energy-efficient, designed for home climate control.
With energy-efficient, remote control, easy installation, this product offers both functionality and style. Its sleek, minimalist design in black or white provides a modern aesthetic and added durability.
Users consistently appreciate , making it highly recommended.
Overall, the XYZ Smart Thermostat has earned positive reviews for its . With its combination of technology and ease of use, it's an excellent choice for home climate control.


In [15]:
import json
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import nltk
import random

# Download the necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Example JSON data
product_json = '''
{
    "product_name": "SuperWidget",
    "keywords": ["innovative", "durable", "affordable"],
    "sentiments": ["positive", "positive", "neutral"]
}
'''

# Load the JSON data
product_details = json.loads(product_json)
product_name = product_details.get('product_name', 'Unknown Product')
keywords = product_details.get('keywords', [])
sentiments = product_details.get('sentiments', [])

# Preprocess the keywords
def preprocess_keywords(keywords):
    processed_keywords = [gensim.utils.simple_preprocess(keyword) for keyword in keywords]
    return processed_keywords

processed_keywords = preprocess_keywords(keywords)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=processed_keywords, vector_size=100, window=5, min_count=1, workers=4)

# Generate product description based on keywords and sentiments
def generate_product_description(product_name, keywords, sentiments):
    positive_templates = [
        f"The {product_name} is highly praised for its {{}}, making it a top choice among consumers.",
        f"Users love the {product_name} for its {{}}, which sets it apart from the competition.",
        f"One of the standout features of the {product_name} is its {{}}, which has received rave reviews."
    ]
    
    neutral_templates = [
        f"The {product_name} offers {{}} features that cater to a wide range of needs.",
        f"With its  {{}} features, the {product_name} provides a balanced performance.",
        f"Consumers appreciate the  {{}} aspects of the {product_name}, which contribute to its versatility."
    ]
    
    negative_templates = [
        f"However, some users have reported issues with the  {{}} of the {product_name}.",
        f"Despite its many strengths, the {product_name} has faced criticism for its  {{}}.",
        f"While generally well-received, the {product_name} has some drawbacks, particularly its  {{}}."
    ]
    
    description = f"The {product_name} is a remarkable product. "
    
    for keyword, sentiment in zip(keywords, sentiments):
        if sentiment == "positive":
            template = random.choice(positive_templates)
            description += template.format(keyword) + " "
        elif sentiment == "neutral":
            template = random.choice(neutral_templates)
            description += template.format(keyword) + " "
        elif sentiment == "negative":
            template = random.choice(negative_templates)
            description += template.format(keyword) + " "
    
    return description.strip()

# Generate product description
description = generate_product_description(product_name, keywords, sentiments)
print(description)

The SuperWidget is a remarkable product. Users love the SuperWidget for its innovative, which sets it apart from the competition. Users love the SuperWidget for its durable, which sets it apart from the competition. Consumers appreciate the  affordable aspects of the SuperWidget, which contribute to its versatility.


[nltk_data] Downloading package punkt to /Users/coder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/coder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import spacy
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from gensim.models import Word2Vec
import numpy as np

# Load Spacy model for dependency parsing
nlp = spacy.load("en_core_web_sm")

# Sample data
keywords = ["energy-efficient", "remote control", "easy installation"]
sentiments = ["positive", "positive", "neutral"]

# Sample product details
product_details = {
    "product_name": "XYZ Smart Thermostat",
    "brand": "EcoLiving",
    "primary_features": ["energy-efficient", "remote control", "easy installation"],
    "brand_info": "Known for eco-friendly, innovative products",
    "intended_use": "home climate control",
    "technical_specs": "Works with Alexa/Google Assistant, real-time temperature adjustments",
    "aesthetic_qualities": "sleek, minimalist design in black or white",
}

# Sample user reviews
user_reviews = [
    "This thermostat was easy to install and saves a lot on my energy bills.",
    "I love the remote control feature—it’s very convenient.",
    "Sleek design and energy-efficient technology make this a great purchase.",
    "A bit tricky to set up at first, but works perfectly afterward.",
]

# Preprocess reviews for Word2Vec
def preprocess(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

all_text = [preprocess(review) for review in user_reviews]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=all_text, vector_size=100, window=5, min_count=1, workers=4)

# Create a dependency parser function
def extract_keywords(text):
    doc = nlp(text)
    return [token.text for token in doc if token.dep_ in ("amod", "nsubj", "dobj")]

# Extract keywords from reviews
extracted_keywords = [extract_keywords(review) for review in user_reviews]

# Flatten the list of keywords
flattened_keywords = [item for sublist in extracted_keywords for item in sublist]

# Vectorize keywords
vectorizer = CountVectorizer()
# Adjust sentiments to match the number of extracted keywords
adjusted_sentiments = []
for i, keywords in enumerate(extracted_keywords):
    if i < len(sentiments):
        adjusted_sentiments.extend([sentiments[i]] * len(keywords))

# Create labels for sentiments
y = np.array([1 if sentiment == "positive" else 0 for sentiment in adjusted_sentiments])

# Train SGDClassifier
# Adjust labels to correspond to each keyword
y_adjusted = np.repeat(y, [len(keyword_list) for keyword_list in extracted_keywords])

sgd_model = make_pipeline(CountVectorizer(), SGDClassifier())
sgd_model.fit(flattened_keywords, y_adjusted)

# Generate product description
def generate_product_description(details, keywords, sentiments, word2vec_model, sgd_model):
    description = f"The {details['product_name']} by {details['brand']} offers {details['primary_features'][0]}, " \
                  f"designed for {details['intended_use']}.\n"

    # Using word vectors to get similar terms for features and feedback
    features = ', '.join(details['primary_features'])
    description += f"With {features}, this product offers both functionality and style. " \
                   f"Its {details['aesthetic_qualities']} provides a modern aesthetic and added durability.\n"
    
    # Summary of positive feedback based on similar words found in reviews
    positive_terms = []
    for keyword in keywords:
        if keyword in word2vec_model.wv:
            positive_terms.extend([sim[0] for sim in word2vec_model.wv.most_similar(keyword, topn=2)])
    positive_summary = ', '.join(set(positive_terms))
    
    description += f"Users consistently appreciate {positive_summary}, making it highly recommended.\n"
    
    # Drawbacks based on review content, if any
    if "tricky" in positive_terms:
        description += "While some users mentioned it can be a bit tricky to set up initially, " \
                       "most agree it doesn't detract from the product's performance.\n"
    
    # Conclusion
    description += f"Overall, the {details['product_name']} has earned positive reviews for " \
                   f"its {positive_summary}. With its combination of technology and ease of use, " \
                   f"it's an excellent choice for {details['intended_use']}."
    
    return description

# Generate and print the product description
contextual_description = generate_product_description(product_details, keywords, sentiments, word2vec_model, sgd_model)
print("Generated Description:", contextual_description)

ValueError: operands could not be broadcast together with shape (10,) (4,)

In [17]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Sample data
keywords = ["energy-efficient", "remote control", "easy installation"]
sentiments = ["positive", "positive", "neutral"]

# Sample product details
product_details = {
    "product_name": "XYZ Smart Thermostat",
    "brand": "EcoLiving",
    "primary_features": ["energy-efficient", "remote control", "easy installation"],
    "brand_info": "Known for eco-friendly, innovative products",
    "intended_use": "home climate control",
    "technical_specs": "Works with Alexa/Google Assistant, real-time temperature adjustments",
    "aesthetic_qualities": "sleek, minimalist design in black or white",
}

# Sample user reviews
user_reviews = [
    "This thermostat was easy to install and saves a lot on my energy bills.",
    "I love the remote control feature—it’s very convenient.",
    "Sleek design and energy-efficient technology make this a great purchase.",
    "A bit tricky to set up at first, but works perfectly afterward.",
]

# Load the LLaMA model and tokenizer
# model_name = "huggingface/llama-7b"  # Replace with the actual model name
model_name = "/Users/coder/.llama/checkpoints/Llama3.2-1B" # follow https://www.llama.com/llama-downloads/ to download the model
# https://llama3-2-lightweight.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoicWU5M2RrcWY4ZmI4dTFqbGpiamtvMnVqIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvbGxhbWEzLTItbGlnaHR3ZWlnaHQubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjY2NjY2OX19fV19&Signature=n0cza537tVWj5EweWh4SCXE1HAzZalZS36QRY-BpkRBHPYZLlOT9ll2tZGZ3SeGvG8yrLdIIrHBNywqazoBK%7En7tmQcE5B6m6WbWna7O-NdkCC0uiVUcr%7E-6Jcto2YUY6jXNLiKaQ7fSMLlex7HH8XnV%7EJrTxkgE3QhQWyxpSdtrFyGB3e7tBbZ5beGh47KhRzDXM%7EJI-Bz6mkLxgOtTSNkpSb4FXuO9UDmvo7nWwx1rSfiMVqTNZ%7E8J3%7EYuH2WfpszEbahPPkqCA4mdSLaKeYGs4iM9YkBs4KzpvglDZ46XgzJVr6nYT2PED51u5dbmGLsYF0KcguQbTAEPeCxmKA__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=2236418840078449
tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=True)
model = AutoModelForCausalLM.from_pretrained(model_name, legacy=True)

# Generate product description based on keywords and sentiments
def generate_product_description(product_details, keywords, sentiments):
    prompt = f"Generate a detailed product description for {product_details['product_name']} by {product_details['brand']} based on the following details:\n"
    prompt += f"Brand Info: {product_details['brand_info']}\n"
    prompt += f"Intended Use: {product_details['intended_use']}\n"
    prompt += f"Technical Specs: {product_details['technical_specs']}\n"
    prompt += f"Aesthetic Qualities: {product_details['aesthetic_qualities']}\n"
    prompt += "Keywords and Sentiments:\n"
    for keyword, sentiment in zip(keywords, sentiments):
        prompt += f"- {keyword}: {sentiment}\n"
    
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=150)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

# Generate product description
description = generate_product_description(product_details, keywords, sentiments)
print(description)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /Users/coder/.llama/checkpoints/Llama3.2-1B.

In [18]:
import spacy
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from gensim.models import Word2Vec
import numpy as np

# Load Spacy model for dependency parsing
nlp = spacy.load("en_core_web_sm")

# Sample data
keywords = ["energy-efficient", "remote control", "easy installation"]
sentiments = ["positive", "positive", "neutral"]

# Sample product details
product_details = {
    "product_name": "XYZ Smart Thermostat",
    "brand": "EcoLiving",
    "primary_features": ["energy-efficient", "remote control", "easy installation"],
    "brand_info": "Known for eco-friendly, innovative products",
    "intended_use": "home climate control",
    "technical_specs": "Works with Alexa/Google Assistant, real-time temperature adjustments",
    "aesthetic_qualities": "sleek, minimalist design in black or white",
}

# Sample user reviews
user_reviews = [
    "This thermostat was easy to install and saves a lot on my energy bills.",
    "I love the remote control feature—it’s very convenient.",
    "Sleek design and energy-efficient technology make this a great purchase.",
    "A bit tricky to set up at first, but works perfectly afterward.",
]

# Preprocess reviews for Word2Vec
def preprocess(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

all_text = [preprocess(review) for review in user_reviews]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=all_text, vector_size=100, window=5, min_count=1, workers=4)

# Create a dependency parser function
def extract_keywords(text):
    doc = nlp(text)
    return [token.text for token in doc if token.dep_ in ("amod", "nsubj", "dobj")]

# Extract keywords from reviews
extracted_keywords = [extract_keywords(review) for review in user_reviews]

# Flatten the list of keywords
flattened_keywords = [item for sublist in extracted_keywords for item in sublist]

# Vectorize keywords
vectorizer = CountVectorizer()

# Adjust sentiments to match the number of extracted keywords
adjusted_sentiments = []
for i, keywords in enumerate(extracted_keywords):
    if i < len(sentiments):
        adjusted_sentiments.extend([sentiments[i]] * len(keywords))

# Ensure y (sentiments labels) has the correct length
y = np.array([1 if sentiment == "positive" else 0 for sentiment in adjusted_sentiments])

# Train SGDClassifier
# Adjust labels to correspond to each keyword
y_adjusted = np.repeat(y, [len(keyword_list) for keyword_list in extracted_keywords])

# Fix: Ensure flattened_keywords matches the length of y_adjusted
flattened_keywords = flattened_keywords[:len(y_adjusted)]

sgd_model = make_pipeline(CountVectorizer(), SGDClassifier())
sgd_model.fit(flattened_keywords, y_adjusted)

# Generate product description
def generate_product_description(details, keywords, sentiments, word2vec_model, sgd_model):
    description = f"The {details['product_name']} by {details['brand']} offers {details['primary_features'][0]}, " \
                  f"designed for {details['intended_use']}.\n"

    # Using word vectors to get similar terms for features and feedback
    features = ', '.join(details['primary_features'])
    description += f"With {features}, this product offers both functionality and style. " \
                   f"Its {details['aesthetic_qualities']} provides a modern aesthetic and added durability.\n"
    
    # Summary of positive feedback based on similar words found in reviews
    positive_terms = []
    for keyword in keywords:
        if keyword in word2vec_model.wv:
            positive_terms.extend([sim[0] for sim in word2vec_model.wv.most_similar(keyword, topn=2)])
    positive_summary = ', '.join(set(positive_terms))
    
    description += f"Users consistently appreciate {positive_summary}, making it highly recommended.\n"
    
    # Drawbacks based on review content, if any
    if "tricky" in positive_terms:
        description += "While some users mentioned it can be a bit tricky to set up initially, " \
                       "most agree it doesn't detract from the product's performance.\n"
    
    # Conclusion
    description += f"Overall, the {details['product_name']} has earned positive reviews for " \
                   f"its {positive_summary}. With its combination of technology and ease of use, " \
                   f"it's an excellent choice for {details['intended_use']}."
    
    return description

# Generate and print the product description
contextual_description = generate_product_description(product_details, keywords, sentiments, word2vec_model, sgd_model)
print("Generated Description:", contextual_description)


ValueError: operands could not be broadcast together with shape (10,) (4,)

In [19]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load a pre-trained model and tokenizer from Hugging Face
model_name = 'gpt2'  # You can also try other models like 'gpt-neo', 'gpt-3', or 't5-small'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Ensure the model is in evaluation mode
model.eval()

# Function to generate a description using the model
def generate_description_with_llm(product_details, user_reviews, model, tokenizer):
    # Combine product details and user reviews as a prompt for the model
    prompt = f"Product Details: {product_details}\nUser Reviews: {user_reviews}\nGenerate a detailed product description:"

    # Encode the prompt text
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)

    # Generate a response using the model
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=300, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7, top_p=0.9, top_k=50)

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Return the generated description part after the prompt
    return generated_text[len(prompt):]

# Example of how you might call the function
product_details = {
    'product_name': 'Smartphone XYZ',
    'brand': 'Brand A',
    'primary_features': ['OLED Display', '5G Connectivity', 'Long Battery Life'],
    'intended_use': 'general use',
    'aesthetic_qualities': 'sleek and modern design'
}

user_reviews = [
    "This phone has an amazing display and great battery life!",
    "I love the 5G connectivity, but the setup was a bit tricky.",
    "Overall, it's a great phone for everyday use, with top-notch performance."
]

# Generate product description
description = generate_description_with_llm(product_details, user_reviews, model, tokenizer)

# Print the generated description
print("Generated Description:", description)

# Generated Description: Product Description: Smartphone Smart Phone is a smart phone that is designed to be used in a wide range of situations. It is built to work with any device, including smartphones, tablets, and smart TVs. The SmartPhone Smart phone is an innovative and versatile device that can be easily used for any purpose.


/Users/coder/Library/Python/3.12/lib/python/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/coder/Library/Python/3.12/lib/python/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a c

Generated Description: 

Product Description: Smartphone Smart Phone is a smart phone that is designed to be used in a wide range of situations. It is built to work with any device, including smartphones, tablets, and smart TVs. The SmartPhone Smart phone is an innovative and versatile device that can be easily used for any purpose.
.


In [20]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
# model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Define the input prompt
prompt = "Artificial intelligence is transforming the world by"

# Tokenize the input text (prompt) to convert it to token IDs
inputs = tokenizer.encode(prompt, return_tensors="pt")

def simple_model(input_ids):
    # A very simple model that just returns the input_ids as logits
    # This is a placeholder for a real model
    logits = torch.nn.functional.one_hot(input_ids, num_classes=tokenizer.vocab_size).float()
    return torch.nn.functional.log_softmax(logits, dim=-1)


class GPT2LikeModel(nn.Module):
    def __init__(self, vocab_size, d_model=768, nhead=12, num_layers=12, d_ff=3072, max_seq_len=512):
        super(GPT2LikeModel, self).__init__()
        
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        
        # Embedding layer
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_embedding = nn.Embedding(max_seq_len, d_model)
        
        # Transformer layers
        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=d_ff) for _ in range(num_layers)
        ])
        
        # Language modeling head (output logits for each token in the vocabulary)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids):
        seq_len = input_ids.size(1)  # sequence length
        
        # Token embeddings
        token_embeddings = self.token_embedding(input_ids)  # Shape: (batch_size, seq_len, d_model)
        
        # Positional embeddings
        positions = torch.arange(0, seq_len, dtype=torch.long, device=input_ids.device).unsqueeze(0)
        positional_embeddings = self.positional_embedding(positions)  # Shape: (1, seq_len, d_model)
        
        # Add token and positional embeddings
        embeddings = token_embeddings + positional_embeddings
        
        # Pass through transformer layers
        x = embeddings
        for transformer in self.transformer_blocks:
            x = transformer(x)
        
        # Language modeling head
        logits = self.lm_head(x)  # Shape: (batch_size, seq_len, vocab_size)
        
        return logits



# Internal Code for generate function
def generate_custom(model, inputs, max_length=100, top_k=50, top_p=0.95, temperature=0.7, no_repeat_ngram_size=2):
    # Initialize the generated sequence as the input tokens
    input_ids = inputs
    generated_ids = input_ids

    # Loop through each token position and generate new tokens
    for _ in range(max_length - len(input_ids[0])):
        # Get the model's output logits for the current input sequence
        # with torch.no_grad():
        #     outputs = simple_model(generated_ids)
        #     logits = outputs[:, -1, :]  # Get logits for the last token
            # logits = outputs.logits[:, -1, :]  # Get logits for the last token

        # Instantiate the model
        vocab_size = tokenizer.vocab_size  # Adjusted to use the tokenizer's vocab size
        model = GPT2LikeModel(vocab_size)
        logits = model.forward(generated_ids)
        
        # Apply temperature scaling
        logits = logits / temperature

        # Apply top-k sampling
        if top_k > 0:
            # Set all tokens outside the top-k to a very low probability
            top_k_values, top_k_indices = torch.topk(logits, top_k)
            logits = torch.full_like(logits, -float("Inf"))
            # logits.scatter_(1, top_k_indices, top_k_values)

        # Apply top-p (nucleus) sampling
        if top_p < 1.0:
            # Sort the logits to compute cumulative probabilities
            sorted_logits, sorted_indices = torch.sort(logits, descending=True)
            cumulative_probs = torch.cumsum(torch.nn.functional.softmax(sorted_logits, dim=-1), dim=-1)
            
            # Filter out tokens with cumulative probability greater than top-p
            sorted_indices_to_keep = cumulative_probs <= top_p
            sorted_indices_to_keep = sorted_indices_to_keep.unsqueeze(0)  # Ensure dimensions match
            logits = torch.full_like(logits, -float("Inf"))
            # logits.scatter_(1, sorted_indices[sorted_indices_to_keep], sorted_logits[sorted_indices_to_keep])

        # Sample from the logits distribution
        probabilities = torch.nn.functional.softmax(logits[:, -1, :], dim=-1)  # Ensure correct dimensions
        next_token_id = torch.multinomial(probabilities, num_samples=1)

        # Append the generated token to the sequence
        generated_ids = torch.cat((generated_ids, next_token_id), dim=-1)

        # Prevent the model from repeating n-grams (this is a very simplified version)
        if no_repeat_ngram_size > 0:
            for i in range(1, no_repeat_ngram_size + 1):
                ngram = generated_ids[0, -i:]
                # Check for repeating n-grams and remove them if necessary (simplified version)
                if any((generated_ids[:, :-i] == ngram).sum(dim=-1) > 0):
                    generated_ids = generated_ids[:, :-1]
                    break

        # Assuming input_ids and targets are defined
        optimizer = Adam(model.parameters(), lr=1e-5)

        # Dummy target (next-token prediction task)
        targets = input_ids[:, 1:].contiguous()  # Shifted by one to predict next token

        # Forward pass
        logits = model(input_ids)

        # Calculate loss (cross-entropy)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print("Loss:", loss.item())

    return generated_ids

# Call the custom generation function
# generated_ids = generate_custom(model, inputs, max_length=100, top_k=50, top_p=0.95, temperature=0.7)
# model = simple_model(inputs)

# Generate text using the custom generation function
generated_ids = generate_custom(GPT2LikeModel, inputs, max_length=100, top_k=50, top_p=0.95, temperature=0.7)

# Decode the generated token IDs back into human-readable text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print the generated text
print("Generated Text:")
print(generated_text)


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [21]:
def generate_text(model, prompt, max_length=50):
    # Convert prompt to token ids
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # Generate tokens one-by-one
    generated_ids = input_ids
    for _ in range(max_length):
        # Forward pass to get logits
        logits = model(generated_ids)
        
        # Get the logits for the last token
        next_token_logits = logits[:, -1, :]
        
        # Sample next token using temperature and top-k sampling (or greedy)
        next_token_id = torch.argmax(next_token_logits, dim=-1)
        
        # Append the token to the sequence
        generated_ids = torch.cat((generated_ids, next_token_id.unsqueeze(-1)), dim=1)
        
        print("Generated:", tokenizer.decode(next_token_id))
        
    # Decode the generated ids into text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

model = GPT2LikeModel(vocab_size=tokenizer.vocab_size)
# Example generation
generated_text = generate_text(model, "The beauty product is", max_length=50)
print(generated_text)


Generated: Earth
Generated: 224
Generated:  Food
Generated:  chimpanzees
Generated:  poisoned
Generated:  Human
Generated:  hugs
Generated:  masters
Generated:  logo
Generated: Choose
Generated:  freak
Generated: Netflix
Generated:  liaison
Generated: Josh
Generated: hat
Generated:  slept
Generated:  giveaway
Generated:  superheroes
Generated:  family
Generated: OB
Generated:  database
Generated:  399
Generated:  pivotal
Generated: itone
Generated: 53
Generated: iggins
Generated:  eleph
Generated: irk
Generated:  Overt
Generated: ars
Generated: Col
Generated:  Challenger
Generated: rw
Generated:  Label
Generated:  particle
Generated:  concerns
Generated: uming
Generated:  259
Generated:  Govern
Generated: igated
Generated:  Bru
Generated:  gust
Generated: ama
Generated: Death
Generated: was
Generated:  moments
Generated:  certify
Generated:  cautiously
Generated: ected
Generated:  timid
The beauty product isEarth224 Food chimpanzees poisoned Human hugs masters logoChoose freakNetflix l

In [ ]:
# RNN to generate text